# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '32'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 2 max_iteration = 5 model_type = base *********
n              iter (since)      4 loss (since)       4 MAE (since) 4 MAE (dev) (since)      dt
1             0.000 (0.000)       0.870 (0.870)       0.438 (0.438)       0.000 (0.000)  15.8 s
2             0.000 (0.000)       0.773 (0.675)       0.359 (0.281)       0.000 (0.000)  27.1 s
4             0.000 (0.000)       1.048 (1.324)       0.508 (0.656)       0.000 (0.000)  50.5 s
8             0.000 (0.000)       0.947 (0.845)       0.441 (0.375)       0.000 (0.000)  1.64 m
16            0.000 (0.000)       0.794 (0.642)       0.359 (0.277)       0.000 (0.000)  3.33 m
32            0.000 (0.000)       0.711 (0.627)       0.335 (0.311)       0.000 (0.000)  6.44 m
64            0.000 (0.000)       0.645 (0.578)       0.307 (0.278)       0.000 (0.000)  12.6 m
128           0.000 (0.000)       0.609 (0.573)       0.289 (0.271)       0.000 (0.000)  25.2 m
256           0.000 (0.000)       0.579 (0.550)       0.279 (0.269)  

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    os.environ['MODEL_TYPE'] = 'base'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '32'
    os.environ['LEARN_BATCH_SIZE'] = '32'
    os.environ['STEP1_ITER'] = '2_augment2'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 2 max_iteration = 5 model_type = base *********
n              iter (since)      4 loss (since)       4 MAE (since) 4 MAE (dev) (since)      nsamps (since)      dt
1             0.000 (0.000)       0.446 (0.446)       0.125 (0.125)       0.000 (0.000)      56.594 (56.594)  1.77 m
2             0.000 (0.000)       0.495 (0.544)       0.203 (0.281)       0.000 (0.000)      32.781 (8.969)  2.57 m
4             0.000 (0.000)       0.456 (0.418)       0.156 (0.109)       0.000 (0.000)      19.992 (7.203)  3.96 m
8             0.000 (0.000)       0.472 (0.489)       0.199 (0.242)       0.000 (0.000)      12.945 (5.898)  6.79 m
16            0.000 (0.000)       0.524 (0.575)       0.234 (0.270)       0.000 (0.000)       9.484 (6.023)  12.4 m
32            0.000 (0.000)       0.527 (0.529)       0.232 (0.230)       0.000 (0.000)       7.737 (5.990)  23.2 m
64            0.000 (0.000)       0.530 (0.533)       0.233 (0.234)       0.000 (0.000)       6.915 (6.092)  44.9 m
128 

# Step 3: Prepare Submission Files

In [ ]:
def prepare_submission_probensemble(*, nvoters, step2_iter, step1_iter, k):
    import json
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedProductRating import dev_loader, test_loader
    from ProgressPrinter import ProgressPrinter
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(8675309)

    dev = dev_loader(batch_size=8)
    test = test_loader(batch_size=8)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')
    t5.load_adapter(f'User_keq{k}_t5base_step2_iter{step2_iter}', 'rhat')
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat", model_id=f'User_keq{k}_t5base_step2_iter{step2_iter}')

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(embeddings, nsamples):
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0], 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape).to(scores.device)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0)

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    print(f'*** step1_iter: {step1_iter} step2_iter: {step2_iter} nvoters {nvoters} ***')
    
    with ProgressPrinter(f'{k} MAE (dev)', f'{k} MAEMLE (dev)', f'{k} MSE (dev)') as printer:
        devgolds, testgolds = [], []
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for isdev, (examples, labels) in interleave(dev, test, sequential=True):
            with torch.no_grad():
                texts_to_embed = [ [ text[:256]
                                     for text in (' '.join(ex['review'].split()), )
                                   ] +
                                   [ text[:256]
                                     for v in ex['profile']
                                     for text in (' '.join(v['text'].split()), )
                                   ]
                                   for ex in examples
                                 ]
                embeddings = torch.cat(inner_batch(func = dev.embed,
                                                   inner_batch_size = 128,
                                                   inputs = (sum(texts_to_embed, []),)
                                                  ),
                                       dim=0)
                splits = cumsum(map(len, texts_to_embed))
                randos = [ randomized_similarity(embeddings[a:b,:], 64) for a, b in zip(splits, splits[1:]) ]
                prompts = [ [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in indices ])
                              for indices in rando.to('cpu').tolist()
                            ]
                            for ex, rando in zip(examples, randos)
                          ]
                rhats = torch.cat(inner_batch(func = rewardpredictor.predict,
                                              inner_batch_size = 128,
                                              inputs = (sum(prompts, []),)
                                             ),
                                  dim=0)
                splits = cumsum(map(len, prompts))
                votingprompts = [ [ prompt[v] for v in torch.topk(rhats[a:b].view(-1), k=min(nvoters, b-a)).indices.to('cpu').tolist() ]
                                    for a, b, prompt in zip(splits, splits[1:], prompts)
                                ]
                predicts = torch.cat(inner_batch(func = taskllm.predict,
                                                 inner_batch_size = 128,
                                                 inputs = (sum(votingprompts, []),)
                                                ),
                                     dim=0)
                splits = cumsum(map(len, votingprompts))
                guesses = torch.cat([ (predicts[a:b,:].logsumexp(dim=0, keepdim=True).exp().cumsum(dim=1) >= 0.5 * (b-a)).long().argmax(dim=1)
                                      for a, b in zip(splits, splits[1:])
                                    ],
                                    dim=0)

                if isdev:
                    targets = [ int(label) - 1 for label in labels ]
                    targets = torch.Tensor(targets).long().to(guesses.device)
                    mae = torch.abs(guesses - targets).float().mean().item()
                    mse = torch.square(guesses - targets).float().mean().item()
                else:
                    mae, maemle, mse = None, None, None

                for ex, guess in zip(examples, guesses):
                    (devgolds if isdev else testgolds).append({ 'id': ex['id'], 'output': f'{1+guess}' })

            printer.addobs(mae, maemle, mse)

        printer.print()
        printer.autoprint = False

        for wut, golds in ( ('dev', devgolds), ('test', testgolds) ):
            with open(f'lamp3u_{wut}golds_t5base_keq{k}_step1_iter{step1_iter}_step2_iter{step2_iter}_nvoters{nvoters}.json', 'w') as jsonfile:
                json.dump({ 'task': 'LaMP_3', 'golds': golds }, jsonfile)
            
for nvoters in [1, 3, 5, 7]:
    prepare_submission_probensemble(k=4, step1_iter='2_augment2', step2_iter='0_augment2', nvoters=nvoters)

*** step1_iter: 2_augment2 step2_iter: 0_augment2 nvoters 1 ***
n          4 MAE (dev) (since)    4 MSE (dev) (since)      dt
1                0.375 (0.375)          0.375 (0.375)  5.19 s
2                0.250 (0.125)          0.250 (0.125)  9.79 s
4                0.219 (0.188)          0.219 (0.188)  17.7 s
8                0.250 (0.281)          0.250 (0.281)  35.4 s
16               0.305 (0.359)          0.383 (0.516)   1.2 m
32               0.281 (0.258)          0.344 (0.305)   2.4 m
64               0.291 (0.301)          0.424 (0.504)  4.71 m
128              0.270 (0.248)          0.357 (0.291)  9.47 m
256              0.253 (0.237)          0.323 (0.288)    19 m
512              0.255 (0.263)          0.325 (0.338)  38.2 m
*** step1_iter: 2_augment2 step2_iter: 0_augment2 nvoters 3 ***
n          4 MAE (dev) (since)    4 MSE (dev) (since)      dt
1                0.375 (0.375)          0.375 (0.375)  5.45 s
2                0.250 (0.125)          0.250 (0.125)  10.2 s
4   

> Hi, 
>
> This is the results of your latest submission to the LaMP benchmark:
>
> LaMP-3:
> {"MAE": 0.2356, "RMSE": 0.5347896782848375}